#Option Pricing and Real Price Comparison

The codes provided code snippets cover various aspects of option pricing, from fetching option chain data to calculating option prices and implied volatility.
The Black-Scholes model is used for European options, while the Binomial Option Pricing Model is used for both European and American options.
Implied volatility calculations allow traders to assess the market's expectation of future volatility.
The code provides a practical approach to option pricing and volatility calculation, making it useful for traders and financial analysts.

# Formula/Function

Import necesaay libararies:
* numpy (np):
a powerful library for numerical computing in Python.
* pandas (pd):
a library for data manipulation and analysis in Python.
* scipy.stats.norm:
a module from the SciPy library that provides functions and classes for working with statistical distributions. Specifically, the norm module deals with the normal (Gaussian) distribution and includes functions like probability density function (pdf), cumulative distribution function (cdf), quantiles, etc.
* yfinance as yf:
Fetch historical market data, stock information, and perform other financial operations using Yahoo Finance's API.
* datetime:
a module provides classes for working with dates and times in Python.

In [19]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf
import pandas as pd
from datetime import datetime

##Black-Schole

**Black-Scholes Option Pricing Model**
This class Black_Scholes calculates option prices and deltas using the Black-Scholes model.

* __init__(self, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility): Initializes the parameters needed for option pricing.
* d1(self): Calculates the d1 parameter used in the Black-Scholes formula.
* d2(self): Calculates the d2 parameter used in the Black-Scholes formula.
* call_option_price(self): Calculates the price of a call option.
* call_option_delta(self): Calculates the delta of a call option.
* put_option_price(self): Calculates the price of a put option.
* put_option_delta(self): Calculates the delta of a put option.

In [20]:
import numpy as np
from scipy.stats import norm

class Black＿Scholes:
    def __init__(self, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility):
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.dividend_yield = dividend_yield
        self.risk_free_rate = risk_free_rate
        self.time_to_maturity = time_to_maturity
        self.volatility = volatility

    def d1(self):
        d1 = (np.log(self.spot_price / self.strike_price) + (self.dividend_yield + (self.volatility ** 2) / 2) * self.time_to_maturity) / (self.volatility * np.sqrt(self.time_to_maturity))
        return d1

    def d2(self):
        d1 = self.d1()
        d2 = d1 - self.volatility * np.sqrt(self.time_to_maturity)
        return d2

    def call_option_price(self):
        d1 = self.d1()
        d2 = self.d2()
        call_option_price = self.spot_price * np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(d1) - self.strike_price * np.exp(-self.risk_free_rate * self.time_to_maturity) * norm.cdf(d2)
        return call_option_price

    def call_option_delta(self):
        d1 = self.d1()
        call_option_delta = np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(d1)
        return call_option_delta

    def put_option_price(self):
        d1 = self.d1()
        d2 = self.d2()
        put_option_price = self.strike_price * np.exp(-self.risk_free_rate * self.time_to_maturity) * norm.cdf(-d2) - self.spot_price * np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(-d1)
        return put_option_price

    def put_option_delta(self):
        d1 = self.d1()
        put_option_delta = -np.exp((self.dividend_yield - self.risk_free_rate) * self.time_to_maturity) * norm.cdf(-d1)
        return put_option_delta

##Binomial Tree

**Binomial Option Pricing Model**
This class BinomialOptionPricing implements the Binomial Option Pricing Model to calculate option prices for both European and American options.

* __init__(self, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps): Initializes the parameters needed for binomial option pricing.
* generate_asset_price(self): Generates the asset price tree.
generate_european_call(self): Calculates the price of a European call option.
* generate_american_call(self): Calculates the price of an American call option.
* generate_european_put(self): Calculates the price of a European put option.
* generate_american_put(self): Calculates the price of an American put option.
* european_call_price(self): Returns the price of a European call option.
* american_call_price(self): Returns the price of an American call option.
* european_put_price(self): Returns the price of a European put option.
* american_put_price(self): Returns the price of an American put option.

In [21]:
import numpy as np
import pandas as pd

class BinomialOptionPricing:
    def __init__(self, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.dividend_yield = dividend_yield
        self.risk_free_rate = risk_free_rate
        self.time_to_maturity = time_to_maturity
        self.volatility = volatility
        self.num_steps = num_steps
        self.delta_t = time_to_maturity / num_steps
        self.u = np.exp(volatility * np.sqrt(self.delta_t))
        self.d = 1 / self.u
        self.r = np.exp(risk_free_rate * self.delta_t)
        self.b = np.exp(dividend_yield * self.delta_t)
        self.q = (self.b - self.d) / (self.u - self.d)
        self.y = self.generate_asset_price()

    def generate_asset_price(self):
        y = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if i == 0 and j == 0:
                    y[i, j] = self.spot_price
                elif i == j:
                    y[i, j] = y[i-1, j-1] * self.d
                elif i < j:
                    y[i, j] = y[i, j-1] * self.u
                else:
                    y[i, j] = 0

        return y

    def generate_european_call(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.y[i, j] - self.strike_price)

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r
            n += 1
        return x

    def generate_american_call(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.y[i, j] - self.strike_price)

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = np.maximum(self.y[i, j] - self.strike_price, (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r)

            n += 1
        return x

    def generate_european_put(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.strike_price - self.y[i, j])

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r
            n += 1
        return x

    def generate_american_put(self):
        x = np.zeros((self.num_steps + 1, self.num_steps + 1))

        for i in range(self.num_steps + 1):
            for j in range(self.num_steps + 1):
                if j == self.num_steps:
                    x[i, j] = max(0, self.strike_price - self.y[i, j])

        n = 1
        for k in range(self.num_steps + 1 - n):
            for i in range(self.num_steps - n + 1):
                for j in range(self.num_steps):
                    if j <= self.num_steps - 1 and i <= self.num_steps - 1 and i <= j:
                        x[i, j] = np.maximum(self.strike_price - self.y[i, j], (self.q * x[i, j+1] + (1 - self.q) * x[i+1, j+1]) / self.r)

            n += 1
        return x

    def european_call_price(self):
      european_call_price = self.generate_european_call()
      return european_call_price[0, 0]

    def american_call_price(self):
      american_call_price = self.generate_american_call()
      return american_call_price[0, 0]

    def european_put_price(self):
      generate_european_put = self.generate_european_put()
      return generate_european_put[0, 0]

    def american_put_price(self):
      generate_american_put = self.generate_american_put()
      return generate_american_put[0, 0]

# Real Option Data

## Grab Data from Yahoo Finance

1. Ticker and option_chain:
Using the yfinance library, these parts of the code fetch option chain data for a given ticker symbol and expiration date. The option chain data includes information on both calls and puts available for trading.
2. Download Risk-Free Rate:
fetches data for Treasury bond yields using yfinance. It downloads the 6-month T-bill, 5-year, 10-year, and 30-year Treasury bond yields. These yields are often used as proxies for risk-free rates in financial modeling.
3. Historical Data:
fetches historical stock data for the past 52 weeks using yfinance. It then calculates the dividends yield rate and percentage returns for the stock over the given period.

In [22]:
import yfinance as yf
import pandas as pd

ticker_symbol = 'SPY'
ticker = yf.Ticker(ticker_symbol)

# Retrieve available expiration dates for options
expiration_dates = ticker.options

# Convert expiration dates to a DataFrame for display
if expiration_dates:
    expiration_dates_df = pd.DataFrame(expiration_dates, columns=['Expire Dates'])
    print(expiration_dates_df)
else:
    print('No expiration dates available.')

print('Number of expiration dates:', len(expiration_dates))

   Expire Dates
0    2024-05-09
1    2024-05-10
2    2024-05-13
3    2024-05-14
4    2024-05-15
5    2024-05-16
6    2024-05-17
7    2024-05-24
8    2024-05-31
9    2024-06-07
10   2024-06-14
11   2024-06-21
12   2024-06-28
13   2024-07-19
14   2024-07-31
15   2024-08-16
16   2024-08-30
17   2024-09-20
18   2024-09-30
19   2024-10-18
20   2024-10-31
21   2024-12-20
22   2024-12-31
23   2025-01-17
24   2025-03-21
25   2025-03-31
26   2025-06-20
27   2025-09-19
28   2025-12-19
29   2026-01-16
30   2026-12-18
Number of expiration dates: 31


In [23]:
import yfinance as yf
import pandas as pd

#expiration_date = '2025-03-21'  # Replace '2024-06-19' with your desired expiration date
expiration_date = expiration_dates[20]

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Retrieve the option chain for the specified expiration date
option_chain = ticker.option_chain(expiration_date)

# Convert the calls and puts data into DataFrames
calls_df = pd.DataFrame(option_chain.calls)
puts_df = pd.DataFrame(option_chain.puts)
expiration_date = pd.to_datetime(expiration_date)
expiration_date

print(f"Total Call Numbers is {len(calls_df)}")
print(f"Total Put Numbers is {len(puts_df)}")

Total Call Numbers is 54
Total Put Numbers is 65


In [24]:
try:
  assert expiration_date > pd.Timestamp(datetime.today().date())
except AssertionError:
  raise ValueError("Change date later than today")

In [25]:
#risk-free
import yfinance as yf
from datetime import datetime

# Define the ticker symbols for Treasury bonds
tickers = ["^IRX", "^FVX", "^TNX", "^TYX"]  # 6-month T-bill, 5-year, 10-year, 30-year

# Fetch data
Risk_Free_Rate = yf.download(tickers, start = datetime.today().date() - pd.Timedelta(days = 7), end=datetime.today().date(), interval="1wk")
# Filter only 'Adj Close' columns
adj_close_columns = [col for col in Risk_Free_Rate.columns if 'Adj Close' in col]
Risk_Free_Rate_adj_close = Risk_Free_Rate[adj_close_columns]
# Clean column names
Risk_Free_Rate_adj_close.columns = [col[1] for col in Risk_Free_Rate_adj_close.columns]
Risk_Free_Rate = Risk_Free_Rate_adj_close.reset_index()
Risk_Free_Rate = Risk_Free_Rate.rename(columns = {"^IRX": "TB13W", "^FVX": "TB5", "^TNX": "TB10", "^TYX": "TB30"})
Risk_Free_Rate["Date"] = pd.to_datetime(Risk_Free_Rate["Date"])
Risk_Free_Rate.iloc[:, 1:] = Risk_Free_Rate.iloc[:, 1:]/100

[*********************100%%**********************]  4 of 4 completed


In [26]:
Risk_Free = Risk_Free_Rate["TB13W"].tail(1).values[0]

In [27]:
difference = expiration_date - pd.Timestamp(datetime.today().date())
difference.days

175

In [28]:
import yfinance as yf

ticker = yf.Ticker(ticker_symbol)

# Retrieve historical data for the past 3 months
historical_data = ticker.history(period= '52wk', interval='1wk')
historical_data = historical_data.reset_index()
historical_data['Date'] = pd.to_datetime(historical_data['Date'])
historical_data['Date'] = historical_data['Date'].dt.strftime('%Y-%m-%d')
historical_data.head(3)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2023-05-08,407.972103,408.622685,403.033608,405.714783,236739400,0.0,0.0,0.0
1,2023-05-15,406.335802,414.714469,404.374218,412.644440,400138800,0.0,0.0,0.0
2,2023-05-22,412.664169,414.763739,404.029203,414.024445,421134200,0.0,0.0,0.0


In [29]:
historical_data['Dividends_Yield'] = historical_data['Dividends']/historical_data['Close']
dividend_yield_rate = historical_data['Dividends_Yield'].mean() * 52
dividend_yield_rate

0.014342595391236949

In [30]:
historical_data['Return'] = historical_data['Close'].pct_change()
historical_data = historical_data.fillna(0)
historical_data.round(3).head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Dividends_Yield,Return
0,2023-05-08,407.972,408.623,403.034,405.715,236739400,0.0,0.0,0.0,0.0,0.000
1,2023-05-15,406.336,414.714,404.374,412.644,400138800,0.0,0.0,0.0,0.0,0.017
2,2023-05-22,412.664,414.764,404.029,414.024,421134200,0.0,0.0,0.0,0.0,0.003
3,2023-05-29,416.006,422.620,410.279,421.812,363259500,0.0,0.0,0.0,0.0,0.019
4,2023-06-05,422.167,425.824,419.742,423.763,362551300,0.0,0.0,0.0,0.0,0.005


* Calculates the standard deviation of the returns, which gives you a measure of the volatility of the stock.
* Adjusts the standard deviation for the fact that you're using weekly data. By dividing by the square root of the number of observations in a year (52 weeks), you're annualizing the volatility.
* Further adjusts the volatility to account for the fact that there are 12 months in a year.

In [31]:
volatility_past = historical_data['Return'].std()/np.sqrt(1/52) * np.sqrt(12)
volatility_past

0.41818166826019676

* Retrieves historical data for the past 1 day (i.e., the most recent trading day).
* Checks if the retrieved data is not empty.
* Extracts the most recent closing price from the retrieved data.
* Prints the spot price for the specified ticker symbol with two decimal places.
* If no real-time data is available, it prints a message indicating that.

In [32]:
import yfinance as yf

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)
# Retrieve the real-time price (most recent price)
real_time_data = ticker.history(period='1d')
# Extract the spot price (most recent closing price)
if not real_time_data.empty:
    spot_price = real_time_data['Close'].iloc[-1]
    print(f"Spot price for {ticker_symbol}: ${spot_price:.2f}")
else:
    print('No real-time data available.')

Spot price for SPY: $517.19


## Call Option

### Use implied volatility to get price

1. Adjusting Calls DataFrame:
* Selects only the necessary columns from the DataFrame.
* Inserts a new column 'Tickers' with the ticker symbol at the beginning of the DataFrame.
* Adds a column 'Spot Price' with the spot price rounded to 2 decimal places.
* Adds a column 'Volatility_Past' with the calculated past volatility.
2. Option Pricing Functions:
* calculate_bs_call_price: Defines a function to calculate the Black-Scholes call option price.
* calculate_bt_call_eu_price: Defines a function to calculate the European call option price using the Binomial Option Pricing Model.
* calculate_bt_call_am_price: Defines a function to calculate the American call option price using the Binomial Option Pricing Model.
3. Option Price Calculation:
* num_steps: Sets the number of steps for the Binomial Option Pricing Model.
* calls_df_price['BS_Price']: Calculates the Black-Scholes call option price for each row using the implied volatility.
* calls_df_price['BT_EU_Price']: Calculates the Binomial European call option price for each row using the implied volatility.
* calls_df_price['BT_AM_Price']: Calculates the Binomial American call option price for each using the implied volatility.
4. Final DataFrame:
* Reorders the columns in the DataFrame to have a better structure for viewing.

In [33]:
calls_df.head(3)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,SPY241031C00360000,2024-05-08 17:36:34+00:00,360.0,163.98,0.0,0.0,0.0,0.0,25.0,0,0.00001,True,REGULAR,USD
1,SPY241031C00390000,2024-05-03 16:51:39+00:00,390.0,129.98,0.0,0.0,0.0,0.0,2.0,0,0.00001,True,REGULAR,USD
2,SPY241031C00400000,2024-04-29 14:46:22+00:00,400.0,120.25,0.0,0.0,0.0,0.0,NaN,0,0.00001,True,REGULAR,USD


In [34]:
calls_df = calls_df[['strike', 'lastPrice', 'bid', 'ask','impliedVolatility']]
calls_df.insert(0, 'Tickers', ticker_symbol)
calls_df['Spot Price'] = spot_price.round(2)
calls_df['Volatility_Past'] = volatility_past
calls_df_price = calls_df.copy()
calls_df_price.head(3)

<ipython-input-34-93eddbb06170>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_df['Spot Price'] = spot_price.round(2)
<ipython-input-34-93eddbb06170>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_df['Volatility_Past'] = volatility_past


,Tickers,strike,lastPrice,bid,ask,impliedVolatility,Spot Price,Volatility_Past
0,SPY,360.0,163.98,0.0,0.0,0.00001,517.19,0.418182
1,SPY,390.0,129.98,0.0,0.0,0.00001,517.19,0.418182
2,SPY,400.0,120.25,0.0,0.0,0.00001,517.19,0.418182


In [35]:
def calculate_bs_call_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility):
    bs = Black＿Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)
    return bs.call_option_price()
def calculate_bt_call_eu_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.european_call_price()
def calculate_bt_call_am_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.american_call_price()

In [36]:
num_steps = 20

In [37]:
# Assuming calls_df is your DataFrame containing options data
calls_df_price['BS_Price'] = calls_df_price.apply(lambda row: calculate_bs_call_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility']), axis=1)
calls_df_price['BS_Price'] = calls_df_price['BS_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
calls_df_price['BT_EU_Price'] = calls_df_price.apply(lambda row: calculate_bt_call_eu_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
calls_df_price['BT_EU_Price'] = calls_df_price['BT_EU_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
calls_df_price['BT_AM_Price'] = calls_df_price.apply(lambda row: calculate_bt_call_am_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
calls_df_price['BT_AM_Price'] = calls_df_price['BT_AM_Price'].round(2)

In [38]:
calls_df_price = calls_df_price[['Tickers','Spot Price', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility', 'Volatility_Past','BS_Price', 'BT_EU_Price', 'BT_AM_Price']]

In [39]:
calls_df_price.head()

,Tickers,Spot Price,strike,lastPrice,bid,ask,impliedVolatility,Volatility_Past,BS_Price,BT_EU_Price,BT_AM_Price
0,SPY,517.19,360.0,163.98,0.0,0.0,0.00001,0.418182,162.59,3.199319e+41,157.19
1,SPY,517.19,390.0,129.98,0.0,0.0,0.00001,0.418182,133.33,2.140654e+41,127.19
2,SPY,517.19,400.0,120.25,0.0,0.0,0.00001,0.418182,123.58,3.135358e+41,117.19
3,SPY,517.19,420.0,108.50,0.0,0.0,0.00001,0.418182,104.08,2.822456e+41,97.19
4,SPY,517.19,450.0,65.25,0.0,0.0,0.00001,0.418182,74.82,2.362953e+41,67.19


### Use volatility from the past to get the price

In [40]:
calls_df_volatility_price = calls_df.copy()

1. Adjusting Calls DataFrame for Volatility Price:
* Creates a copy of the original DataFrame for calculating option prices using the past volatility.
2. Option Price Calculation using Past Volatility:
* calls_df_volatility_price['BS_Price']: Calculates the Black-Scholes call option price for each row in the DataFrame using the past volatility.
* calls_df_volatility_price['BT_EU_Price']: Calculates the European call option price for each row in the DataFrame using the past volatility.
* calls_df_volatility_price['BT_AM_Price']: Calculates the American call option price for each row in the DataFrame using the past volatility.
3. Final DataFrame for Volatility Price:
* Reorders the columns in the DataFrame to have a better structure for viewing.

In [41]:
# Assuming calls_df is your DataFrame containing options data
calls_df_volatility_price['BS_Price'] = calls_df_volatility_price.apply(lambda row: calculate_bs_call_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['Volatility_Past']), axis=1)
calls_df_volatility_price['BS_Price'] = calls_df_volatility_price['BS_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
calls_df_volatility_price['BT_EU_Price'] = calls_df_volatility_price.apply(lambda row: calculate_bt_call_eu_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['Volatility_Past'], num_steps), axis=1)
calls_df_volatility_price['BT_EU_Price'] = calls_df_volatility_price['BT_EU_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
calls_df_volatility_price['BT_AM_Price'] = calls_df_volatility_price.apply(lambda row: calculate_bt_call_am_price(row, spot_price, row['strike'], Risk_Free - dividend_yield_rate, Risk_Free, difference.days/365, row['Volatility_Past'], num_steps), axis=1)
calls_df_volatility_price['BT_AM_Price'] = calls_df_volatility_price['BT_AM_Price'].round(2)

In [42]:
calls_df_volatility_price = calls_df_volatility_price[['Tickers','Spot Price', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility', 'Volatility_Past','BS_Price', 'BT_EU_Price', 'BT_AM_Price']]

In [43]:
calls_df_volatility_price.head()

,Tickers,Spot Price,strike,lastPrice,bid,ask,impliedVolatility,Volatility_Past,BS_Price,BT_EU_Price,BT_AM_Price
0,SPY,517.19,360.0,163.98,0.0,0.0,0.00001,0.418182,167.98,167.97,167.97
1,SPY,517.19,390.0,129.98,0.0,0.0,0.00001,0.418182,143.18,143.10,143.10
2,SPY,517.19,400.0,120.25,0.0,0.0,0.00001,0.418182,135.36,134.91,134.91
3,SPY,517.19,420.0,108.50,0.0,0.0,0.00001,0.418182,120.45,120.84,120.84
4,SPY,517.19,450.0,65.25,0.0,0.0,0.00001,0.418182,100.07,99.74,99.74


## Put Option

### Use implied volatility to get price

1. Adjusting Puts DataFrame:
* Selects only the necessary columns from the DataFrame.
* Inserts a new column 'Tickers' with the ticker symbol at the beginning of the DataFrame.
* Adds a column 'Spot Price' with the spot price rounded to 2 decimal places.
* Adds a column 'Volatility_Past' with the calculated past volatility.
2. Option Pricing Functions for Puts:
* calculate_bs_put_price: Defines a function to calculate the Black-Scholes put option price.
* calculate_bt_put_eu_price: Defines a function to calculate the European put option price using the Binomial Option Pricing Model.
* calculate_bt_put_am_price: Defines a function to calculate the American put option price using the Binomial Option Pricing Model.
3. Option Price Calculation for Puts:
* puts_df_price['BS_Price']: Calculates Black-Scholes put option price for each row using the implied volatility.
* puts_df_price['BT_EU_Price']: Calculates the Binomial European put option price for each row using the implied volatility.
* puts_df_price['BT_AM_Price']: Calculates the Binomial American put option price for each row using the implied volatility
4. Final DataFrame for Put Options:
*  Reorders the columns in the DataFrame to have a better structure for viewing.

In [44]:
puts_df = puts_df[['strike', 'lastPrice', 'bid', 'ask','impliedVolatility']]
puts_df.insert(0, 'Tickers', ticker_symbol)
puts_df.head()

,Tickers,strike,lastPrice,bid,ask,impliedVolatility
0,SPY,360.0,0.98,0.0,0.0,0.125009
1,SPY,370.0,1.08,0.0,0.0,0.125009
2,SPY,380.0,1.30,0.0,0.0,0.062509
3,SPY,390.0,1.42,0.0,0.0,0.062509
4,SPY,400.0,1.63,0.0,0.0,0.062509


In [45]:
puts_df['Spot Price'] = spot_price.round(2)
puts_df['Volatility_Past'] = volatility_past
puts_df_price = puts_df.copy()
puts_df_price.head(3)

<ipython-input-45-eea48457bca4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puts_df['Spot Price'] = spot_price.round(2)
<ipython-input-45-eea48457bca4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  puts_df['Volatility_Past'] = volatility_past


,Tickers,strike,lastPrice,bid,ask,impliedVolatility,Spot Price,Volatility_Past
0,SPY,360.0,0.98,0.0,0.0,0.125009,517.19,0.418182
1,SPY,370.0,1.08,0.0,0.0,0.125009,517.19,0.418182
2,SPY,380.0,1.30,0.0,0.0,0.062509,517.19,0.418182


In [46]:
def calculate_bs_put_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility):
    bs = Black＿Scholes(spot_price, strike_price, Risk_Free - dividend_yield, risk_free_rate, time_to_maturity, volatility)
    return bs.put_option_price()
def calculate_bt_put_eu_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, Risk_Free - dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.european_put_price()
def calculate_bt_put_am_price(row, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps):
    bt = BinomialOptionPricing(spot_price, strike_price, Risk_Free - dividend_yield,risk_free_rate, time_to_maturity, volatility, num_steps)
    return bt.american_put_price()

In [47]:
num_steps = 40

In [48]:
# Assuming calls_df is your DataFrame containing options data
puts_df_price['BS_Price'] = puts_df_price.apply(lambda row: calculate_bs_put_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility']), axis=1)
puts_df_price['BS_Price'] = puts_df_price['BS_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
puts_df_price['BT_EU_Price'] = puts_df_price.apply(lambda row: calculate_bt_put_eu_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
puts_df_price['BT_EU_Price'] = puts_df_price['BT_EU_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
puts_df_price['BT_AM_Price'] = puts_df_price.apply(lambda row: calculate_bt_put_am_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['impliedVolatility'], num_steps), axis=1)
puts_df_price['BT_AM_Price'] = puts_df_price['BT_AM_Price'].round(2)

In [49]:
puts_df_price = puts_df_price[['Tickers','Spot Price', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility', 'Volatility_Past','BS_Price', 'BT_EU_Price', 'BT_AM_Price']]

In [50]:
puts_df_price.head()

,Tickers,Spot Price,strike,lastPrice,bid,ask,impliedVolatility,Volatility_Past,BS_Price,BT_EU_Price,BT_AM_Price
0,SPY,517.19,360.0,0.98,0.0,0.0,0.125009,0.418182,0.0,0.0,0.0
1,SPY,517.19,370.0,1.08,0.0,0.0,0.125009,0.418182,0.0,0.0,0.0
2,SPY,517.19,380.0,1.30,0.0,0.0,0.062509,0.418182,0.0,0.0,0.0
3,SPY,517.19,390.0,1.42,0.0,0.0,0.062509,0.418182,0.0,0.0,0.0
4,SPY,517.19,400.0,1.63,0.0,0.0,0.062509,0.418182,0.0,0.0,0.0


### Use volatility from the past to get the price

In [51]:
puts_df_volatility_price = puts_df.copy()

1. Adjusting Puts DataFrame for Volatility Price:
* puts_df_volatility_price = puts_df.copy(): Creates a copy of the original DataFrame for calculating option prices using the past volatility.
2. Option Price Calculation using Past Volatility for Puts:
* puts_df_volatility_price['BS_Price']: Calculates the Black-Scholes put option price for each row in the DataFrame using the past volatility.
* puts_df_volatility_price['BT_EU_Price']: Calculates the Binomial European put option price for each row using the past volatility.
* puts_df_volatility_price['BT_AM_Price']: Calculates the Binomial American put option price for each row using the past volatility.
3. Final DataFrame for Volatility Price for Put Options:
* puts_df_volatility_price: Reorders the columns in the DataFrame to have a better structure for viewing.

In [52]:
puts_df_volatility_price.head()

,Tickers,strike,lastPrice,bid,ask,impliedVolatility,Spot Price,Volatility_Past
0,SPY,360.0,0.98,0.0,0.0,0.125009,517.19,0.418182
1,SPY,370.0,1.08,0.0,0.0,0.125009,517.19,0.418182
2,SPY,380.0,1.30,0.0,0.0,0.062509,517.19,0.418182
3,SPY,390.0,1.42,0.0,0.0,0.062509,517.19,0.418182
4,SPY,400.0,1.63,0.0,0.0,0.062509,517.19,0.418182


In [53]:
# Assuming calls_df is your DataFrame containing options data
puts_df_volatility_price['BS_Price'] = puts_df_volatility_price.apply(lambda row: calculate_bs_put_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['Volatility_Past']), axis=1)
puts_df_volatility_price['BS_Price'] = puts_df_volatility_price['BS_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
puts_df_volatility_price['BT_EU_Price'] = puts_df_volatility_price.apply(lambda row: calculate_bt_put_eu_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['Volatility_Past'], num_steps), axis=1)
puts_df_volatility_price['BT_EU_Price'] = puts_df_volatility_price['BT_EU_Price'].round(2)

# Assuming calls_df is your DataFrame containing options data
puts_df_volatility_price['BT_AM_Price'] = puts_df_volatility_price.apply(lambda row: calculate_bt_put_am_price(row, spot_price, row['strike'], dividend_yield_rate, Risk_Free, difference.days/365, row['Volatility_Past'], num_steps), axis=1)
puts_df_volatility_price['BT_AM_Price'] = puts_df_volatility_price['BT_AM_Price'].round(2)

In [54]:
puts_df_volatility_price = puts_df_volatility_price[['Tickers','Spot Price', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility', 'Volatility_Past','BS_Price', 'BT_EU_Price', 'BT_AM_Price']]

In [55]:
puts_df_volatility_price.head()

,Tickers,Spot Price,strike,lastPrice,bid,ask,impliedVolatility,Volatility_Past,BS_Price,BT_EU_Price,BT_AM_Price
0,SPY,517.19,360.0,0.98,0.0,0.0,0.125009,0.418182,5.39,5.25,5.32
1,SPY,517.19,370.0,1.08,0.0,0.0,0.125009,0.418182,6.67,6.74,6.81
2,SPY,517.19,380.0,1.30,0.0,0.0,0.062509,0.418182,8.15,8.22,8.31
3,SPY,517.19,390.0,1.42,0.0,0.0,0.062509,0.418182,9.85,9.71,9.82
4,SPY,517.19,400.0,1.63,0.0,0.0,0.062509,0.418182,11.78,11.80,11.95


# Run Option Value

## Black-Schole

### Call Option

1. Define Parameters:
* spot_price: Current stock price.
* strike_price: Option strike price.
* time_to_maturity: Time to option expiration in years.
* volatility: Volatility of the underlying stock.
* risk_free_rate: Risk-free interest rate.
* dividend: Dividend yield rate.
* dividend_yield: Dividend yield calculated as the difference between risk-free rate and dividend yield rate.
2. Create Black-Scholes Option Object:
* Black_Schole_Option = Black＿Scholes(...): Initialize a Black-Scholes option object with the provided parameters.

#### Calculate Price by Manual Input of Data


In [56]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 116.75  # Current stock price
strike_price = 50.0   # Option strike price
time_to_maturity = difference.days/365  # Time to option expiration in years
#time_to_maturity = 3/12
volatility = 0.689704 #np.sqrt(0.1)  # Volatility of the underlying stock
risk_free_rate = Risk_Free # Risk-free interest rate
#risk_free_rate = 0.0425 # Risk-free interest rate
dividend =  dividend_yield_rate
#dividend =  0.0125
dividend_yield =  risk_free_rate - dividend

In [57]:
Black_Schole_Option = Black＿Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)

In [58]:
print(f"d1 is {Black_Schole_Option.d1():.3f}\nd2 is {Black_Schole_Option.d2():.3f}")
print(f"N(d1) is {norm.cdf(Black_Schole_Option.d1()):.3f}\nN(d2) is {norm.cdf(Black_Schole_Option.d2()):.3f}")
print(f"Call Value is {Black_Schole_Option.call_option_price():.3f}\nCall Delta is {Black_Schole_Option.call_option_delta():.3f}")

d1 is 2.053
d2 is 1.575
N(d1) is 0.980
N(d2) is 0.942
Call Value is 67.676
Call Delta is 0.973


#### Reverse calculation to get volatility

1. Parameters:
* initial_guess: Initial guess for the volatility.
* tolerance: Tolerance level for convergence.
* max_iterations: Maximum number of iterations for convergence.
2. implied_call_volatility Function:
* Calculates the Black-Scholes call price using the provided volatility.
* Uses the Newton-Raphson method to iteratively refine the volatility until the calculated option price matches the observed call price within the specified tolerance.
* Returns the implied volatility.
3. Usage:
* Sets the observed option price.
* Calculates the implied volatility using implied_call_volatility.
* Rounds the result to 3 decimal places and prints it.

In [59]:
initial_guess = 0.5
tolerance = 0.001
max_iterations = 100000

In [60]:
def implied_call_volatility(call_price, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def black_scholes_call_price(volatility):
        Black_Schole_Option.volatility = volatility
        return Black_Schole_Option.call_option_price()

    # Newton-Raphson method
    vol = initial_guess
    for i in range(max_iterations):
        option_price = black_scholes_call_price(vol)
        vega = Black_Schole_Option.call_option_delta() * Black_Schole_Option.spot_price * np.exp(-Black_Schole_Option.dividend_yield * Black_Schole_Option.time_to_maturity) / np.sqrt(2 * np.pi * Black_Schole_Option.time_to_maturity)

        if np.abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

call_price = 67.32
implied_vol = implied_call_volatility(call_price)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility Call:", rounded_implied_vol)

Implied Volatility Call: 0.56


### Put Option

#### Calculate Price by Manual Input of Data

Similiar to previous section

In [61]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 516.57  # Current stock price
strike_price = 395.0   # Option strike price
#time_to_maturity = difference.days/365  # Time to option expiration in years
time_to_maturity = 3/12
volatility = 0.421576 #np.sqrt(0.1)  # Volatility of the underlying stock
#risk_free_rate = Risk_Free # Risk-free interest rate
risk_free_rate = 0.0425 # Risk-free interest rate
#dividend =  dividend_yield_rate
dividend =  0.0125
dividend_yield =  risk_free_rate - dividend

In [62]:
Black_Schole_Option = Black＿Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)

In [63]:
print(f"d1 is {Black_Schole_Option.d1():.3f}\nd2 is {Black_Schole_Option.d2():.3f}")
print(f"N(d1) is {norm.cdf(Black_Schole_Option.d1()):.3f}\nN(d2) is {norm.cdf(Black_Schole_Option.d2()):.3f}")
print(f"Put Value is {Black_Schole_Option.put_option_price():.3f}\nPut Delta is {Black_Schole_Option.put_option_delta():.3f}")

d1 is 1.128
d2 is 0.836
N(d1) is 0.870
N(d2) is 0.798
Put Value is 11.117
Put Delta is -0.129


#### Reverse calculation to get volatility

Similar to previous section but use put option price

In [64]:
initial_guess = 0.5
tolerance = 0.001
max_iterations = 10000

In [65]:
def implied_put_volatility(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def black_scholes_put_price(volatility):
        bs = Black_Scholes(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility)
        return bs.put_option_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = black_scholes_put_price(vol)
        vega = (black_scholes_put_price(vol * 1.001) - black_scholes_put_price(vol)) / (0.001 * vol)

        if abs(option_price - put_price) < tolerance:
            return vol

        vol -= (option_price - put_price) / vega

    return np.nan  # If not found within tolerance

put_price = 0.04
implied_vol = implied_put_volatility(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility Put:", rounded_implied_vol)


Implied Volatility Put: 0.15


## Binomial Tree

### Call Option

1. Define Parameters:
* spot_price: Current stock price.
* strike_price: Option strike price.
* time_to_maturity: Time to option expiration in years.
* volatility: Volatility of the underlying stock.
* risk_free_rate: Risk-free interest rate.
* dividend_yield: Dividend yield rate.
* num_steps: Number of steps in the binomial tree.
2. Create Binomial Option Pricing Object:
Initialize a BinomialOptionPricing object with the provided parameters.
3. Generate Asset Prices:
Generate the asset price tree using generate_asset_price() method and store it in a DataFrame.
4. Generate Option Prices:
* Generate the European call option prices using generate_european_call() method and store them in a DataFrame.
* Generate the American call option prices using generate_american_call() method and store them in a DataFrame.
5. DataFrames for Visualization:
Convert the generated data into DataFrames for better visualization. Round the values to two decimal places.

#### Calculate Price by Manual Input of Data



In [92]:
import numpy as np

spot_price = 3790 # Current stock price
strike_price = 3825   # Option strike price
#time_to_maturity = difference.days/365  # Time to option expiration in years
time_to_maturity = 3/12
volatility = 0.5 #np.sqrt(0.1)  # Volatility of the underlying stock
#risk_free_rate = Risk_Free # Risk-free interest rate
risk_free_rate = 0.03 # Risk-free interest rate
#dividend =  dividend_yield_rate
dividend =  0.02
dividend_yield =  risk_free_rate - dividend

num_steps = 10

binomial_tree_option = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)

In [93]:
Asset_Price_df = pd.DataFrame(binomial_tree_option.generate_asset_price())
Asset_Price_df = Asset_Price_df.round(2)
#Asset_Price_df.replace(0, '', inplace=True)Asset_Price_df

In [94]:
euro_call_df = pd.DataFrame(binomial_tree_option.generate_european_call())
euro_call_df = euro_call_df.round(2)
#euro_call_df.replace(0, '', inplace=True)
euro_call_df

,0,1,2,3,4,5,6,7,8,9,10
0,358.59,525.53,749.65,1037.88,1391.38,1803.99,2264.65,2765.11,3307.24,3894.50,4530.63
1,0.00,203.87,317.90,482.71,710.68,1009.73,1378.25,1802.58,2265.01,2765.97,3308.64
2,0.00,0.00,98.13,165.12,271.44,433.64,668.53,985.68,1375.20,1802.47,2265.36
3,0.00,0.00,0.00,35.97,66.49,121.01,215.85,374.59,624.91,979.89,1374.66
4,0.00,0.00,0.00,0.00,7.65,15.89,33.01,68.55,142.38,295.73,614.23
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [95]:
am_call_df = pd.DataFrame(binomial_tree_option.generate_american_call())
am_call_df = am_call_df.round(2)
#am_call_df.replace(0, '', inplace=True)
am_call_df

,0,1,2,3,4,5,6,7,8,9,10
0,358.6,525.56,749.70,1037.99,1391.59,1804.37,2265.36,2766.39,3308.64,3895.50,4530.63
1,0.0,203.87,317.91,482.72,710.70,1009.77,1378.33,1802.75,2265.36,2766.39,3308.64
2,0.0,0.00,98.13,165.12,271.44,433.64,668.53,985.68,1375.20,1802.47,2265.36
3,0.0,0.00,0.00,35.97,66.49,121.01,215.85,374.59,624.91,979.89,1374.66
4,0.0,0.00,0.00,0.00,7.65,15.89,33.01,68.55,142.38,295.73,614.23
5,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [96]:
print(f"Binomial European Call Value: {binomial_tree_option.european_call_price(): .3f}\nBinomial American Call Value: {binomial_tree_option.american_call_price(): .3f}")

Binomial European Call Value:  358.586
Binomial American Call Value:  358.601


#### Reverse calculation to get volatility


In [71]:
num_steps = 20
initial_guess = 0.5
tolerance = 0.001
max_iterations = 100

In [72]:
def implied_volatility_call_eu_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps=num_steps, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def binomial_tree_call_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.european_call_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_call_price(vol)
        vega = (binomial_tree_call_price(vol * 1.001) - binomial_tree_call_price(vol)) / (0.001 * vol)

        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

call_price = 122.78
implied_vol = implied_volatility_call_eu_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility European Call (Binomial Tree):", rounded_implied_vol)

<ipython-input-21-ce8e056a9fd7>:15: RuntimeWarning: divide by zero encountered in scalar divide
  self.d = 1 / self.u
<ipython-input-21-ce8e056a9fd7>:18: RuntimeWarning: invalid value encountered in scalar divide
  self.q = (self.b - self.d) / (self.u - self.d)
<ipython-input-21-ce8e056a9fd7>:31: RuntimeWarning: invalid value encountered in scalar multiply
  y[i, j] = y[i, j-1] * self.u


Implied Volatility European Call (Binomial Tree): nan


In [73]:
def implied_volatility_call_am_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps=num_steps, initial_guess=initial_guess, tolerance=tolerance, max_iterations=max_iterations):
    def binomial_tree_call_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.american_call_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_call_price(vol)
        vega = (binomial_tree_call_price(vol * 1.001) - binomial_tree_call_price(vol)) / (0.001 * vol)

        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

call_price = 122.78
implied_vol = implied_volatility_call_eu_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility American Call (Binomial Tree):", rounded_implied_vol)

<ipython-input-21-ce8e056a9fd7>:15: RuntimeWarning: divide by zero encountered in scalar divide
  self.d = 1 / self.u
<ipython-input-21-ce8e056a9fd7>:18: RuntimeWarning: invalid value encountered in scalar divide
  self.q = (self.b - self.d) / (self.u - self.d)
<ipython-input-21-ce8e056a9fd7>:31: RuntimeWarning: invalid value encountered in scalar multiply
  y[i, j] = y[i, j-1] * self.u


Implied Volatility American Call (Binomial Tree): nan


### Put Option

Similar to previous section but for put option

#### Calculate Price by Manual Input of Data

In [74]:
import numpy as np
# Example usage:
spot_price = 516.57  # Current stock price
strike_price = 395.0   # Option strike price
#time_to_maturity = difference.days/365  # Time to option expiration in years
time_to_maturity = 3/12
volatility = 0.421576 #np.sqrt(0.1)  # Volatility of the underlying stock
#risk_free_rate = Risk_Free # Risk-free interest rate
risk_free_rate = 0.0425 # Risk-free interest rate
#dividend =  dividend_yield_rate
dividend =  0.0125
dividend_yield =  risk_free_rate - dividend

num_steps = 20

binomial_tree_option = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)

In [75]:
Asset_Price_df = pd.DataFrame(binomial_tree_option.generate_asset_price())
Asset_Price_df = Asset_Price_df.round(2)
#Asset_Price_df.replace(0, '', inplace=True)
#Asset_Price_df.head(num_steps+1)
Asset_Price_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,516.57,551.41,588.61,628.31,670.69,715.93,764.21,815.76,870.78,929.52,...,1059.14,1130.58,1206.84,1288.24,1375.13,1467.89,1566.90,1672.58,1785.40,1905.83
1,0.00,483.93,516.57,551.41,588.61,628.31,670.69,715.93,764.21,815.76,...,929.52,992.22,1059.14,1130.58,1206.84,1288.24,1375.13,1467.89,1566.90,1672.58
2,0.00,0.00,453.35,483.93,516.57,551.41,588.61,628.31,670.69,715.93,...,815.76,870.78,929.52,992.22,1059.14,1130.58,1206.84,1288.24,1375.13,1467.89
3,0.00,0.00,0.00,424.70,453.35,483.93,516.57,551.41,588.61,628.31,...,715.93,764.21,815.76,870.78,929.52,992.22,1059.14,1130.58,1206.84,1288.24
4,0.00,0.00,0.00,0.00,397.87,424.70,453.35,483.93,516.57,551.41,...,628.31,670.69,715.93,764.21,815.76,870.78,929.52,992.22,1059.14,1130.58


In [76]:
euro_put_df = pd.DataFrame(binomial_tree_option.generate_european_put())
euro_put_df = euro_put_df.round(2)
#euro_put_df.replace(0, '', inplace=True)
euro_put_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,10.76,6.46,3.52,1.70,0.70,0.23,0.05,0.01,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,14.94,9.30,5.28,2.67,1.16,0.40,0.10,0.01,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.00,20.41,13.19,7.82,4.14,1.88,0.70,0.19,0.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.00,0.00,27.41,18.41,11.38,6.32,3.03,1.19,0.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.00,0.00,0.00,36.15,25.23,16.29,9.49,4.82,2.01,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
am_put_df = pd.DataFrame(binomial_tree_option.generate_american_put())
am_put_df = am_put_df.round(2)
#am_put_df.replace(0, '', inplace=True)
am_put_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,10.89,6.52,3.55,1.71,0.70,0.23,0.05,0.01,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,15.12,9.40,5.33,2.69,1.16,0.40,0.10,0.01,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.00,20.68,13.34,7.88,4.16,1.89,0.70,0.19,0.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.00,0.00,27.79,18.63,11.49,6.36,3.05,1.19,0.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.00,0.00,0.00,36.70,25.55,16.46,9.57,4.85,2.02,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
print(f"Binomial European Put Value: {binomial_tree_option.european_put_price(): .3f}\nBinomial American Put Value: {binomial_tree_option.american_put_price(): .3f}")

Binomial European Put Value:  10.764
Binomial American Put Value:  10.887


#### Reverse calculation to get volatility

In [79]:
num_steps = 20
initial_guess = 0.5
tolerance = 1e-3
max_iterations = 100

In [80]:
def implied_volatility_put_am_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps = num_steps, initial_guess = initial_guess, tolerance = tolerance, max_iterations = max_iterations):
    def binomial_tree_put_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.european_put_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_put_price(vol)
        vega = (binomial_tree_put_price(vol * 1.001) - binomial_tree_put_price(vol)) / (0.001 * vol)
        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

put_price = 0.0
implied_vol = implied_volatility_put_am_binomial_tree(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility European Put (Binomial Tree):", rounded_implied_vol)

Implied Volatility European Put (Binomial Tree): 0.109


In [81]:
def implied_volatility_put_am_binomial_tree(call_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, num_steps = num_steps, initial_guess = initial_guess, tolerance = tolerance, max_iterations = max_iterations):
    def binomial_tree_put_price(volatility):
        bt = BinomialOptionPricing(spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity, volatility, num_steps)
        return bt.american_put_price()

    # Newton-Raphson method
    vol = initial_guess
    for _ in range(max_iterations):
        option_price = binomial_tree_put_price(vol)
        vega = (binomial_tree_put_price(vol * 1.001) - binomial_tree_put_price(vol)) / (0.001 * vol)
        if abs(option_price - call_price) < tolerance:
            return vol

        vol -= (option_price - call_price) / vega

    return np.nan  # If not found within tolerance

put_price = 0.04
implied_vol = implied_volatility_put_am_binomial_tree(put_price, spot_price, strike_price, dividend_yield, risk_free_rate, time_to_maturity)
rounded_implied_vol = round(float(implied_vol), 3)
print("Implied Volatility Americna Put (Binomial Tree):", rounded_implied_vol)

Implied Volatility Americna Put (Binomial Tree): 0.152
